# Notebook 3 – Build Real Factors from Market Data

This notebook shows how to build SC, CS, INNOV, and TAIL
using yfinance and the `src.factors` helpers.


In [1]:
import pandas as pd
import numpy as np

from src.factors import (
    download_prices,
    to_returns,
    build_sc_factor,
    build_spread_factor,
    build_innovation_factor,
    build_tail_factor,
)

tickers = ["IWM", "HYG", "IEF", "QQQ", "^VIX"]
prices = download_prices(tickers, start="2005-01-01")
prices.tail()

Ticker,HYG,IEF,IWM,QQQ,^VIX
Date,,,,,
2025-11-06,80.260002,96.650002,240.350006,611.669983,19.500000
2025-11-07,80.330002,96.690002,241.610001,609.739990,19.080000
2025-11-10,80.599998,96.559998,244.029999,623.229980,17.600000
2025-11-11,80.690002,96.860001,244.240005,621.570007,17.280001
2025-11-12,80.529999,96.879997,243.639999,621.080017,17.510000


## Convert to monthly returns

In [2]:
rets = to_returns(prices, freq="M")
rets.tail()

/Users/bz/Downloads/Unified_Privates_Model_HL_Extended_20251113_050130/notebooks/src/factors.py:55: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  logp = logp.resample(freq).last()


Ticker,HYG,IEF,IWM,QQQ,^VIX
Date,,,,,
2025-07-31,0.001357,-0.005957,0.016545,0.023948,-0.000598
2025-08-31,0.010993,0.016347,0.069458,0.009494,-0.084839
2025-09-30,0.008821,0.006490,0.031296,0.052367,0.058171
2025-10-31,-0.000111,0.007088,0.017494,0.046696,0.068829
2025-11-30,0.001740,0.003454,-0.010574,-0.012783,0.004006


## Build SC, CS, INNOV, and TAIL factors

In [3]:
sc = build_sc_factor(rets, sc_ticker="IWM")
cs = build_spread_factor(rets["HYG"], rets["IEF"])
innov = build_innovation_factor(rets["QQQ"], sc)

# For TAIL, first build ΔVIX from monthly VIX levels
vix = prices["^VIX"].resample("M").last().pct_change() * 100.0
tail = build_tail_factor(vix)

factors_real = pd.concat([sc, cs, innov, tail], axis=1).dropna()
factors_real.columns = ["SC", "CS", "INNOV", "TAIL"]
factors_real.tail()

/var/folders/55/623vgtzj3hn7ffs5kj635y1c0000gn/T/ipykernel_91715/2369985394.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vix = prices["^VIX"].resample("M").last().pct_change() * 100.0


,SC,CS,INNOV,TAIL
Date,,,,
2025-07-31,0.016545,-0.001627,0.018006,0.000000
2025-08-31,0.069458,0.005470,0.003639,0.000000
2025-09-30,0.031296,0.004420,0.008465,0.141497
2025-10-31,0.017494,-0.004580,0.007107,0.205002
2025-11-30,-0.010574,-0.002315,0.000615,0.000000
